<a href="https://colab.research.google.com/github/heber-augusto/udacity-generative-ai-nanodegree/blob/main/personalized-real-state-agent/personalized-real-state-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Personalized Real Estate Agent
Project from Udacity Generative AI Nano Degreet

The task is to develop an innovative application named "HomeMatch". This application leverages large language models (LLMs) and vector databases to transform standard real estate listings into personalized narratives that resonate with potential buyers' unique preferences and needs.

## Read OpenAI API KEY

In [1]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('openai_api_key')

## Libraries instalations

In [41]:
!pip install langchain openai==0.28 chromadb tiktoken
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.3 MB/s eta 0:00:00


## Libraries imports

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document


## Generating Real Estate Listings with an LLM

In [7]:
model_name="gpt-4o"
temperature = 1.2

chat_llm = ChatOpenAI(
    model_name=model_name,
    temperature=temperature,
    max_tokens = 4096)

prompt_template = """
Generate at least 10 real estate listings to produce descriptions of various properties. An example of a listing might be:
-----------
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

messages = [
    HumanMessage(content=prompt_template),
]

#output = chat_llm(messages)

#fd = open('listings.txt', 'w')
#fd.write(output.content)
#fd.close()


content="Sure, here are 10 imaginary real estate listings:\n\n1. -----------\nNeighborhood: Maplewood Estate\nPrice: $1,250,000\nBedrooms: 5\nBathrooms: 4\nHouse Size: 3,500 sqft\n\nDescription: Step into luxury at this exquisite Maplewood Estate residence. Boasting 5 bedrooms and 4 bathrooms, this home features a grand staircase, spacious living rooms, and a gourmet kitchen with top-of-the-line stainless steel appliances. The master suite includes a private balcony overlooking the immaculately landscaped backyard, ideal for entertaining with its built-in BBQ area and heated pool. Don't miss out on experiencing elegance and comfort in this spacious family home.\n\nNeighborhood Description: Known for its tree-lined streets and highly rated schools, Maplewood Estate offers a serene suburban lifestyle with a touch of class. Enjoy leisurely walks through the Meadow View Park or indulge in some retail therapy at Maplewood Mall. The community organizes regular family-centric events, making i

### Read file from repository and put the content inside a list

In [29]:
import requests
response = requests.get(
    'https://raw.githubusercontent.com/heber-augusto/udacity-generative-ai-nanodegree/main/personalized-real-state-agent/files/listings.txt')

### Remove the first element from the split and the content at the end of each generated listing

In [36]:

listings = []
for prop in response.text.split('-----------')[1:]:
  listings.append(prop[:-5])

## Semantic Search

## Creating a Vector Database and Storing Listings

In [42]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores import Chroma

docs = [Document(page_content=x) for x in listings]

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)

## Testing Semantic Search of Listings Based on query

In [47]:
query = "I want a house with 3 or more bedrooms inside a calm neigboorhood"
similar_docs = db.similarity_search(
    query,
    k=3)

similar_docs

[Document(page_content='Neighborhood: Cedar Pines\nPrice: $525,000\nBedrooms: 3\nBathrooms: 2.5\nHouse Size: 2,300 sqft\n\nDescription: Welcome to Cedar Pines where this charming 3-bedroom, 2.5-bathroom home awaits. Featuring an open floor plan, a cozy fireplace, and a sunroom that provides ample light, this home combines comfort and elegance. The spacious master bedroom includes an en-suite bath with a soaking tub and walk-in closet. The landscaped backyard offers enough space for customizable gardening or entertainment.\n\nNeighborhood Description: Cedar Pines is a serene neighborhood residing amongst natural woodlands, offering myriad outdoor activities including hiking trails and bird watching. The community emphasizes sustainable living while providing access to top-tier schools and local craft shops.'),
 Document(page_content='Neighborhood: Riverwalk\nPrice: $695,000\nBedrooms: 3\nBathrooms: 3\nHouse Size: 2,200 sqft\n\nDescription: Discover this Riverwalk beauty with 3 bedrooms 

In [48]:
print(listings[0])


Neighborhood: Maplewood Estate
Price: $1,250,000
Bedrooms: 5
Bathrooms: 4
House Size: 3,500 sqft

Description: Step into luxury at this exquisite Maplewood Estate residence. Boasting 5 bedrooms and 4 bathrooms, this home features a grand staircase, spacious living rooms, and a gourmet kitchen with top-of-the-line stainless steel appliances. The master suite includes a private balcony overlooking the immaculately landscaped backyard, ideal for entertaining with its built-in BBQ area and heated pool. Don't miss out on experiencing elegance and comfort in this spacious family home.

Neighborhood Description: Known for its tree-lined streets and highly rated schools, Maplewood Estate offers a serene suburban lifestyle with a touch of class. Enjoy leisurely walks through the Meadow View Park or indulge in some retail therapy at Maplewood Mall. The community organizes regular family-centric events, making it an idyllic place for growing families.
